In [5]:
import pandas as pd
import os,sys
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime,date,timedelta
from itertools import product
from sklearn.metrics import mean_squared_error

os.chdir('C:/Users/arpit.goel/Documents/Projects/Kaggle/15.RecruitRestarauntPrediction')
store={}
store['air_reserve']=pd.read_csv('01.RawData/air_reserve.csv',parse_dates=['visit_datetime','reserve_datetime'])
store['air_store_info']=pd.read_csv('01.RawData/air_store_info.csv',index_col=['air_store_id'])
store['air_visit_data']=pd.read_csv('01.RawData/air_visit_data.csv',parse_dates=['visit_date'],index_col=['air_store_id','visit_date'])
store['date_info']=pd.read_csv('01.RawData/date_info.csv',parse_dates=['calendar_date'],index_col=['calendar_date'])
store['hpg_reserve']=pd.read_csv('01.RawData/hpg_reserve.csv',parse_dates=['visit_datetime','reserve_datetime'])
store['hpg_store_info']=pd.read_csv('01.RawData/hpg_store_info.csv',index_col=['hpg_store_id'])
store['store_id_relation']=pd.read_csv('01.RawData/store_id_relation.csv')
store['hpg_reserve'] = pd.merge(store['hpg_reserve'], store['store_id_relation'], how='inner', on=['hpg_store_id'])

df=pd.read_csv('01.RawData/sample_submission.csv')
df['air_store_id']=df['id'].map(lambda x: '_'.join(x.split('_')[:2]))
df['visit_date']=pd.to_datetime(df['id'].map(lambda x: x.split('_')[2]))
df.set_index(['air_store_id','visit_date'],inplace=True)
store['sample_submission']=df


In [6]:
for df in ['air_reserve','hpg_reserve']:
    store[df]['visit_datetime'] = store[df]['visit_datetime'].dt.date
    store[df]['reserve_datetime'] = store[df]['reserve_datetime'].dt.date
    store[df]['reserve_datetime_diff'] = store[df].apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
    tmp1 = store[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs1', 'reserve_visitors':'rv1'})
    tmp2 = store[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs2', 'reserve_visitors':'rv2'})
    store[df] = pd.merge(tmp1, tmp2, how='inner', on=['air_store_id','visit_date'])


In [ ]:
store['air_visit_data']['dow'] = store['air_visit_data']['visit_date'].dt.dayofweek
store['air_visit_data']['year'] = store['air_visit_data']['visit_date'].dt.year
store['air_visit_data']['month'] = store['air_visit_data']['visit_date'].dt.month
store['air_visit_data']['visit_date'] = store['air_visit_data']['visit_date'].dt.date

store['sample_submission']['dow'] = store['sample_submission']['visit_date'].dt.dayofweek
store['sample_submission']['year'] = store['sample_submission']['visit_date'].dt.year
store['sample_submission']['month'] = store['sample_submission']['visit_date'].dt.month
store['sample_submission']['visit_date'] = store['sample_submission']['visit_date'].dt.date
